# Keras Tuner

A hyperparameter tuner for [Keras](https://keras.io), specifically for `tf.keras` with TensorFlow 2.0.

Full documentation and tutorials available on the [Keras Tuner website.](https://keras-team.github.io/keras-tuner/)

## Keras Tuner CIFAR10 example

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


TensorFlow 2.x selected.


In [0]:
!pip install -U keras-tuner

In [0]:
import kerastuner as kt
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
def build_model(hp):
  inputs = tf.keras.Input(shape=(32, 32, 3))
  x = inputs
  for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
    filters = hp.Int('filters_' + str(i), 32, 256, step=32)
    for _ in range(2):
      x = tf.keras.layers.Convolution2D(
        filters, kernel_size=(3, 3), padding='same')(x)
      x = tf.keras.layers.BatchNormalization()(x)
      x = tf.keras.layers.ReLU()(x)
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
      x = tf.keras.layers.MaxPool2D()(x)
    else:
      x = tf.keras.layers.AvgPool2D()(x)
  x = tf.keras.layers.GlobalAvgPool2D()(x)
  x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
  x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
  outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)
  model.compile(
    optimizer=tf.keras.optimizers.Adam(
      hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  return model

In [0]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2)

In [0]:
data = tfds.load('cifar10')
train_ds, test_ds = data['train'], data['test']

In [0]:
def standardize_record(record):
  return tf.cast(record['image'], tf.float32) / 255., record['label']

In [0]:
train_ds = train_ds.map(standardize_record).batch(64).shuffle(10000)
test_ds = test_ds.map(standardize_record).batch(64)

In [0]:
tuner.search(train_ds,
             validation_data=test_ds,
             epochs=30,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

Epoch 1/2
782/782 [==============================] - 1830s 2s/step - loss: 1.7662 - accuracy: 0.3137 - val_loss: 1.4316 - val_accuracy: 0.4563
Epoch 2/2
782/782 [==============================] - 1822s 2s/step - loss: 1.3603 - accuracy: 0.5016 - val_loss: 1.6363 - val_accuracy: 0.4732


Epoch 1/2
782/782 [==============================] - 1477s 2s/step - loss: 1.2706 - accuracy: 0.5425 - val_loss: 1.0188 - val_accuracy: 0.6365
Epoch 2/2
782/782 [==============================] - 1474s 2s/step - loss: 0.8145 - accuracy: 0.7173 - val_loss: 0.8702 - val_accuracy: 0.6962


Epoch 1/2
782/782 [==============================] - 946s 1s/step - loss: 1.3803 - accuracy: 0.5004 - val_loss: 1.1807 - val_accuracy: 0.5847
Epoch 2/2
782/782 [==============================] - 937s 1s/step - loss: 0.8880 - accuracy: 0.6946 - val_loss: 1.2167 - val_accuracy: 0.5781


Epoch 1/2
    327/Unknown - 1283s 4s/step - loss: 1.8628 - accuracy: 0.2883

In [0]:
best_model = tuner.get_best_models(1)[0]
best_model

In [0]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
best_hyperparameters